# Frame Transformation

In [177]:
import numpy as np

In [160]:
def global_to_local(traj_global, dx, dy, theta):
    """
    convert coordinates from global frame to robot (local) frame
    traj_global: array of humans' trajectories (traj_len, num_of_human, 2) in the global frame
    dx: x-coordinate of robot in global frame
    dy: y-coordinate of robot in global frame
    theta: theta of robot in global frame
    :return: array of humans' trajectories (traj_len, num_of_human, 2) in the local (robot) frame
    """

    T_w_r = np.array([[np.cos(theta), - np.sin(theta), dx],
                      [np.sin(theta), np.cos(theta), dy],
                      [0, 0, 1]])  # 2D transformation matrix

    # T ^ r _ w
    T_r_w = np.linalg.inv(T_w_r)
    # add the homogeneous coordinate
    traj_global_hom = np.concatenate([traj_global, np.ones((traj_global.shape[0], traj_global.shape[1], 1))], axis=2)
    traj_local_hom = np.tensordot(traj_global_hom, T_r_w, axes=([2], [1]))
    traj_local = traj_local_hom[:,:,:-1]   # remove the homogeneous coordinate
    return traj_local

In [167]:
def A_to_B(A_p, A_x_B, A_y_B, A_theta_B):
    """
    convert coordinate from frame A to frame B
    A_p: coordinate in frame A (x, y)
    A_x_B: x-coordinate of frame B in frame A
    A_y_B: y-coordinate of frame B in frame A
    A_theta_B: theta of frame B in frame A
    :return: coordinate in frame B (x, y)
    """

    theta = A_theta_B
    A_T_B = np.array([[np.cos(theta), -np.sin(theta), A_x_B],
                         [np.sin(theta), np.cos(theta), A_y_B],
                         [0, 0, 1]])  # 2D transformation matrix

    B_T_A = np.linalg.inv(A_T_B)

    # Transform the point into the new coordinate frame
    # B_p = B_T_A * A_p
    B_p = np.dot(B_T_A, np.hstack((A_p, 1)))

    return B_p[:-1]   # remove the homogeneous coordinate

In [168]:
# Define the original coordinates in the old coordinate frame
traj = np.array([[[2, 3], [4, 5]],
                 [[2, 3], [5, 6]],
                 [[10, 11], [12, 13]],
                 [[3, 4], [6, 8]]])

In [173]:
global_to_local(traj, dx=2, dy=3, theta=np.pi/2)

array([[[  0.,   0.],
        [  2.,  -2.]],

       [[  0.,   0.],
        [  3.,  -3.]],

       [[  8.,  -8.],
        [ 10., -10.]],

       [[  1.,  -1.],
        [  5.,  -4.]]])

In [171]:
A_to_B(traj[3,1], A_x_B=2, A_y_B=3, A_theta_B=np.pi/2)

array([ 5., -4.])

# Absolute coordinate system to Relative coordinate system

In [174]:
import numpy as np

def abs_to_relative(abs_traj):
    """
    Inputs:
    - abs_traj: numpy of shape (seq_len, batch, 2)
    Outputs:
    - rel_traj: numpy of shape (seq_len, batch, 2)
    """
    rel_traj = np.zeros(abs_traj.shape)
    rel_traj[1:, :, :] = abs_traj[1:, :, :] - abs_traj[:-1, :, :]
    return rel_traj

In [175]:
# seq_len, batch, 2
abs_traj = np.array([[[1, 2], [4, 5]],
                     [[2, 3], [5, 6]],
                     [[3, 4], [6, 8]]])

In [176]:
abs_to_relative(abs_traj)

array([[[0., 0.],
        [0., 0.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 2.]]])

# Local Map Generation

In [178]:
6 * np.sqrt(2)

8.485281374238571

In [179]:
1/.2

5.0

# SocialGAN

In [ ]:
import sys
# setting path
sys.path.append('../sgan')

from predictor import socialGAN

In [ ]:
traj_predictor = socialGAN(model_path='../sgan/models/sgan-p-models/eth_8_model.pt')